In [12]:
import pandas as pd
from scipy import sparse
import pandas as pd
import numpy as np
from numpy import var
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random
from matplotlib import pyplot as plt
from scipy.stats import norm
import numpy as np
from fitter import Fitter, get_common_distributions, get_distributions
from scipy import stats
from scipy.stats import norm
from sklearn.linear_model import LinearRegression

#load data
ratings = pd.read_csv('dataset/ratings.csv')
moviesPartial = pd.read_csv('dataset/movies.csv')
moviesFull = pd.read_csv('dataset/movie_dataset.csv')

#drop duplicates
moviesFull.drop_duplicates(subset = 'title', inplace=True)

#drop columns that do nothing
ratings = ratings.drop(['timestamp'], axis = 1)
moviesPartial = moviesPartial.drop(['genres'],axis=1)

#all the users are to be tested
userRatings = []
for i in range(1,611):
    userRatings.append(ratings[ratings['userId'] ==i])

#remove the year and keep the name
moviesPartial['title'] = moviesPartial['title'].map(lambda string: string[:-7])

#drop duplicates
moviesPartial.drop_duplicates(subset = 'title', inplace=True)

#helper functions:
def get_title_from_movieId(tmp):
    return moviesPartial[moviesPartial["movieId"]== tmp]["title"].to_string(index=False)

def checkTitle(tmp):
    if(tmp not in list(moviesFull["title"])):
        #The word "the" is being apended to the ends of titles
        tmp = tmp[-3:] +" "+ tmp[:-5]
        return moviesFull[moviesFull["title"] == tmp]["title"].to_string(index=False)
    else:
        return moviesFull[moviesFull["title"] == tmp]["title"].to_string(index=False)



#used to filter out movies that are not in both data sets
newUserTitles = []
newUserRatings = []
for userRating in userRatings:
    temp1 = []
    temp2 = []
    for rating, id in zip(list(userRating["rating"]), list(userRating["movieId"])):
        title = get_title_from_movieId(id)
        checked = checkTitle(title)
        if(checked != 'Series([], )'):
            temp1.append(checked)
            temp2.append(rating)
    newUserTitles.append(temp1)
    newUserRatings.append(temp2)
    


#this might lead to a referencce error
#populate infos; a list of data frames
infos = []
for user in newUserTitles:
    i = 0
    tempInfo = pd.DataFrame(columns = list(moviesFull.columns))
    for name in user:
        if(len(moviesFull[moviesFull["title"] == name]) !=0):
            tempInfo.loc[i] = moviesFull[moviesFull["title"] == name].iloc[0]
        i+=1
    infos.append(tempInfo)
    

features = ['keywords','cast','genres','director']


for info in infos:
    for feature in features:
        info[feature] = info[feature].fillna('')

colNumbers = []
for feature in features:   
    colNumbers.append(infos[0].columns.get_loc(feature))

fullDataList =[]


for frame in infos:
    fullDataList.append(frame.values.tolist())

combinedFeatures = []

for item in fullDataList:
    tmp = []
    for row in item:
        tmp.append(row[colNumbers[0]]+" "+row[colNumbers[1]]+" "+row[colNumbers[2]]+" "+row[colNumbers[3]])
    combinedFeatures.append(tmp)


cv = CountVectorizer() 
count_matrix = cv.fit_transform(combinedFeatures[0])


lst = [count_matrix[0]]*(count_matrix.shape[1]-1)


print(count_matrix[1:count_matrix.shape[1]][0:5])
print(lst[0:5])

cosine_sim = cosine_similarity(X = count_matrix[1:count_matrix.shape[1]]
,Y = lst)

#print(cosine_sim)

# index =0
# for item in combinedFeatures:
#     if(index == 4):
#         break
#     cv = CountVectorizer() 
#     count_matrix = cv.fit_transform(item)
#     testData = count_matrix[0]
#     cosine_sim = cosine_similarity(count_matrix[1:len(testData)],count_matrix[0])
#     index+=1




#cosine_similarity sim function computes more then is needed at the moment

#cutome?








  (0, 847)	1
  (0, 1208)	1
  (0, 1456)	1
  (0, 349)	2
  (0, 942)	1
  (0, 1395)	1
  (0, 105)	1
  (0, 567)	1
  (0, 222)	1
  (0, 277)	1
  (0, 1099)	1
  (0, 805)	1
  (0, 1157)	1
  (0, 1129)	1
  (0, 1163)	1
  (0, 437)	1
  (0, 347)	1
  (0, 1464)	1
  (0, 207)	1
  (0, 1348)	1
  (1, 347)	1
  (1, 1464)	1
  (1, 371)	1
  (1, 201)	2
  (1, 1207)	1
  :	:
  (3, 56)	1
  (3, 903)	1
  (3, 675)	1
  (3, 1552)	1
  (4, 1469)	1
  (4, 767)	1
  (4, 1301)	1
  (4, 150)	1
  (4, 1)	1
  (4, 271)	1
  (4, 671)	1
  (4, 1538)	1
  (4, 919)	1
  (4, 865)	1
  (4, 1037)	1
  (4, 755)	1
  (4, 838)	1
  (4, 704)	1
  (4, 1258)	1
  (4, 485)	1
  (4, 1401)	1
  (4, 15)	1
  (4, 980)	1
  (4, 263)	1
  (4, 772)	1
[<1x1616 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>, <1x1616 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>, <1x1616 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row

ValueError: setting an array element with a sequence.